In [96]:
import pandas as pd
import numpy as np
from scipy.stats import gaussian_kde
import statsmodels.api as sm
import json
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import os
from datetime import datetime

In [79]:
raw_data = pd.read_csv('empresasEafit.csv')
raw_data.rename(columns=lambda x: x.strip(), inplace=True)
raw_data = raw_data[~raw_data['Macrosector'].isin(['0', 'No', 'No informa', 'SI', 'Si'])]
raw_data.head()

,Razón social,Macrosector,"Tipo de propiedad (Privada, Pública, Mixta)",¿Es empresa familiar? Si/No,¿Cotiza en bolsa? Si/No,¿Multinacional? Si/No,Año de fundación,País de base principal (nacionalidad),Ingresos operacionales,Variable,Valor,Bloque,Nombre Pilar,Ponderación Materialidad,Valoración,valoracionPonderada
0,Aliar,Agropecuario,Privada,No,No,No,2007,Colombia,0.97802211500000003,¿Cuál es el porcentaje de proveeduría local?,No informa,Impacto económico y gobernanza,"Impacto Económico, Social y Comunitario",14%,0.000000,0.000000
1,Alimentos Carnicos (Grupo Nutresa),Manufactura,Privada,No,Si,Si,1920,Colombia,2.844632077,¿Cuál es el porcentaje de proveeduría local?,0.80000000000000004,Impacto económico y gobernanza,"Impacto Económico, Social y Comunitario",11%,84.210526,8.864266
2,Alpina,Manufactura,Privada,No,No,Si,1945,Colombia,2.1764613960000001,¿Cuál es el porcentaje de proveeduría local?,0.91000000000000003,Impacto económico y gobernanza,"Impacto Económico, Social y Comunitario",11%,95.789474,10.083102
3,Amarilo,Construcción,Privada,Si,No,No,1993,Colombia,2.1863011939999999,¿Cuál es el porcentaje de proveeduría local?,0.53000000000000003,Impacto económico y gobernanza,"Impacto Económico, Social y Comunitario",14%,53.000000,7.571429
4,ARA,Comercio,Privada,No,No,Si,2012,Colombia,13.83118268,¿Cuál es el porcentaje de proveeduría local?,0,Impacto económico y gobernanza,"Impacto Económico, Social y Comunitario",12%,0.000000,0.000000


In [80]:
def save_chart_as_html(fig, filename):
    """
    Guarda una figura de Plotly como un archivo HTML autocontenido en la carpeta 'charts'.
    """
    # Generar el HTML del gráfico
    html1 = pio.to_html(fig, full_html=False, include_plotlyjs='cdn')
    
    # Guardar el archivo
    filepath = os.path.join('charts', filename)
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(html1)

# Function to wrap long text labels
def wrap_text(text, max_length=25):
    """
    Wrap text to multiple lines if it exceeds max_length characters.
    """
    if len(text) <= max_length:
        return text
    
    words = text.split()
    lines = []
    current_line = []
    current_length = 0
    
    for word in words:
        # Check if adding this word would exceed the limit
        if current_length + len(word) + len(current_line) > max_length and current_line:
            lines.append(' '.join(current_line))
            current_line = [word]
            current_length = len(word)
        else:
            current_line.append(word)
            current_length += len(word)
    
    if current_line:
        lines.append(' '.join(current_line))
    
    return '<br>'.join(lines)


# **Parte 1: Panorama General del Desempeño Corporativo**

## 1. Radar de Competitividad por Pilar y Macrosector

In [81]:
# Encadenar las operaciones de pandas para simplificar el proceso:
# 1. Agrupar por Macrosector y Nombre Pilar, calculando la media.
radar_final_df = (
    raw_data.groupby(['Macrosector', 'Nombre Pilar'])['valoracionPonderada']
    .mean()
    .reset_index()
    .pivot_table(
        index='Nombre Pilar',
        columns='Macrosector',
        values='valoracionPonderada'
    )
    .fillna(0)
    .reset_index()
    .rename(columns={'Nombre Pilar': 'category'})
)

categories = radar_final_df['category'].tolist()
macrosectores = radar_final_df.drop(columns='category').columns.tolist()

# Wrap category labels
wrapped_categories = [wrap_text(cat) for cat in categories]

fig = go.Figure()

# Create color palette for different macrosectors
colors = px.colors.qualitative.Set1[:len(macrosectores)]

for i, macrosector in enumerate(macrosectores):
    values = radar_final_df[macrosector].values
    fig.add_trace(go.Scatterpolar(
        r=values,
        theta=wrapped_categories,  # Use wrapped categories
        fill='toself',
        name=macrosector,
        line=dict(color=colors[i % len(colors)]),
        fillcolor=colors[i % len(colors)],
        opacity=0.6
    ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, max(radar_final_df.drop(columns='category').max()) * 1.1],  # Dynamic range
            tickmode='linear',
            tick0=0,
            dtick=5
        ),
        angularaxis=dict(
            tickmode='array',
            tickvals=list(range(len(categories))),
            ticktext=wrapped_categories,  # Use wrapped categories
            tickfont=dict(size=10),  # Smaller font for wrapped text
            rotation=0  # Keep labels horizontal for better readability
        )
    ),
    showlegend=True,
    title={
        'text': '1. Radar de Competitividad por Pilar y Macrosector',
        'x': 0.5,
        'xanchor': 'center'
    }
)
fig.show()
save_chart_as_html(fig, '01_radar_macroeconomic.html')


## 2. Mapa de calor de Desempeño Empresarial

In [82]:
# 1. Pivotar la tabla para agregar los valores.
#    - Se agrupa por 'Razón social' y 'Nombre Pilar'.
#    - Se calcula la SUMA de 'valoracionPonderada' como indica la guía.
#    - Se rellenan los valores nulos con 0.
heatmap_pivot = raw_data.pivot_table(
    index='Razón social',
    columns='Nombre Pilar',
    values='valoracionPonderada',
    aggfunc='sum',
    fill_value=0
)

heatmap_pivot.sort_values(by='Razón social', inplace=True)
wrapped_pilars = [wrap_text(col) for col in heatmap_pivot.columns]
# Crear mapa de calor con Plotly
heatmap_fig = go.Figure(data=go.Heatmap(
    z=heatmap_pivot.values,
    x=wrapped_pilars,
    y=heatmap_pivot.index,
    colorscale='blues',
    colorbar=dict(title='Valoración Ponderada')
))

heatmap_fig.update_layout(
    title='2. Mapa de Calor de Desempeño Empresarial',
    xaxis_title='Pilares',
    yaxis_title='Empresas',
    width=1000,  # Ajustar el ancho del gráfico
    height=800  # Ajustar la altura del gráfico
)

heatmap_fig.show()
save_chart_as_html(heatmap_fig, '02_heatmap_performance.html')

## 3. Diagrama de Violín de la Dispersión del Desempeño Sectorial

In [83]:
# 1. Calcular el puntaje total (suma de valoracionPonderada) para cada empresa.
total_scores = raw_data.groupby(['Razón social', 'Macrosector'])['valoracionPonderada'].sum().reset_index()
total_scores.rename(columns={'valoracionPonderada': 'puntaje_total'}, inplace=True)

# 2. Eliminar Razón social y limpiar Macrosector '0' 'No' 'No informa' 'SI' 'Si'
total_scores.drop(columns='Razón social', inplace=True)


# 3. Crear un violin plot para cada Macrosector.
violin_fig = px.violin(total_scores, y='puntaje_total', x='Macrosector', box=True, points='all',
               color='Macrosector', title='3. Violin Plot de Puntajes por Macrosector',
               labels={'puntaje_total': 'Puntaje Total', 'Macrosector': 'Macrosector'})
violin_fig.update_traces(meanline_visible=True)
violin_fig.update_layout(
    xaxis_title='Macrosector',
    yaxis_title='Puntaje Total'
)

violin_fig.show()
save_chart_as_html(violin_fig, '03_violin_plot_performance.html')

## 4. Treemap de Impacto: Sostenibilidad y Peso Económico

In [84]:
raw_data['valoracionPonderada'] = pd.to_numeric(raw_data['valoracionPonderada'], errors='coerce').fillna(0)
raw_data['Ingresos operacionales'] = pd.to_numeric(raw_data['Ingresos operacionales'], errors='coerce').fillna(0)
# 2. Agregación de Datos por Empresa
# El dataset original tiene múltiples filas por empresa (una por cada 'Variable').
# Necesitamos una única fila por empresa con sus datos agregados.
company_agg_data = raw_data.groupby('Razón social').agg(
    # Se suma la 'valoracionPonderada' para obtener el puntaje total de sostenibilidad.
    Puntaje_Total_Sostenibilidad=('valoracionPonderada', 'sum'),
    # Se toma el primer valor de 'Ingresos operacionales' y 'Macrosector',
    # ya que estos deberían ser constantes para cada empresa.
    Ingresos_Operacionales=('Ingresos operacionales', 'first'),
    Macrosector=('Macrosector', 'first')
).reset_index()

# Para una visualización más efectiva, filtramos empresas con ingresos nulos o negativos.
company_agg_data = company_agg_data[company_agg_data['Ingresos_Operacionales'] > 0]

# 3. Creación del Gráfico Treemap con Plotly Express
# Plotly Express permite crear figuras complejas con una sintaxis sencilla.
fig_treemap = px.treemap(
    company_agg_data,
    # La jerarquía define cómo se anidan los rectángulos.
    # Se añade una raíz común ("Todas las Empresas") para una mejor navegación.
    path=[px.Constant("Todas las Empresas"), 'Macrosector', 'Razón social'],
    # El tamaño de cada rectángulo será proporcional a los ingresos.
    values='Ingresos_Operacionales',
    # El color de cada rectángulo se mapea al puntaje de sostenibilidad.
    color='Puntaje_Total_Sostenibilidad',
    hover_data={
        'Macrosector': True,
        'Ingresos_Operacionales': ':.2f', # Formato de dos decimales para el hover
        'Puntaje_Total_Sostenibilidad': ':.2f'
    },
    # Se elige una escala de color intuitiva: Rojo (bajo) -> Amarillo (medio) -> Verde (alto).
    color_continuous_scale='blues',
    # Se establece el punto medio de la escala de color en el promedio ponderado del puntaje,
    # lo que da una mejor representación visual de quién está por encima o por debajo del promedio.
    color_continuous_midpoint=np.average(company_agg_data['Puntaje_Total_Sostenibilidad'], weights=company_agg_data['Ingresos_Operacionales'])
)

# 4. Personalización y Estilo del Gráfico
fig_treemap.update_layout(
    title_text='<b>Treemap de Impacto: Sostenibilidad y Peso Económico</b><br><sup>El tamaño representa los Ingresos Operacionales, el color el Desempeño en Sostenibilidad</sup>',
    title_x=0.5, # Centra el título del gráfico
    margin = dict(t=60, l=25, r=25, b=25), # Ajusta los márgenes
    font=dict(
        family="Arial, sans-serif",
        size=14,
        color="black"
    )
)

# Se personaliza la información que aparece al pasar el cursor sobre los rectángulos (hover).
# Esto mejora la legibilidad y presenta la información de forma clara y concisa.
fig_treemap.update_traces(
    hovertemplate='<b>%{label}</b><br><br>' +
                  'Macrosector: %{customdata[0]}<br>' +
                  'Ingresos Operacionales (Tamaño): %{value:,.2f}<br>' +
                  'Puntaje Sostenibilidad (Color): %{color:.2f}<extra></extra>' # <extra></extra> oculta información extra de plotly
)

# Muestra el gráfico interactivo.
fig_treemap.show()
# Guarda el gráfico como un archivo HTML autocontenido.
save_chart_as_html(fig_treemap, '04_treemap_sustainability.html')

# **Parte 2: Análisis Relacional y de Composición Jerárquica**

## 5. Diagrama de Burbujas: Sostenibilidad vs. Antigüedad Empresarial

In [85]:
# 1. Preparación y Manejo de Datos
# Asegurarse que 'Año de fundación' sea numérico.
raw_data['Año de fundación'] = pd.to_numeric(raw_data['Año de fundación'], errors='coerce')

# Reutilizamos la agregación anterior y añadimos el año de fundación.
# Usamos 'first' ya que el año de fundación es constante por empresa.
bubble_chart_data = raw_data.groupby('Razón social').agg(
    Puntaje_Total_Sostenibilidad=('valoracionPonderada', 'sum'),
    Ingresos_Operacionales=('Ingresos operacionales', 'first'),
    Macrosector=('Macrosector', 'first'),
    Ano_Fundacion=('Año de fundación', 'first')
).reset_index()

# Limpieza de datos para el gráfico: eliminar filas sin año de fundación o sin ingresos.
bubble_chart_data.dropna(subset=['Ano_Fundacion', 'Ingresos_Operacionales'], inplace=True)
bubble_chart_data = bubble_chart_data[bubble_chart_data['Ingresos_Operacionales'] > 0]
bubble_chart_data['Ano_Fundacion'] = bubble_chart_data['Ano_Fundacion'].astype(int)

# 2. Creación del Gráfico de Burbujas con Plotly Express
fig_bubble = px.scatter(
    bubble_chart_data,
    x="Ano_Fundacion",
    y="Puntaje_Total_Sostenibilidad",
    size="Ingresos_Operacionales", # El tamaño de la burbuja representa los ingresos
    color="Macrosector",          # El color de la burbuja representa el macrosector
    hover_name="Razón social",    # Muestra el nombre de la empresa al pasar el cursor
    size_max=60,                  # Controla el tamaño máximo de la burbuja más grande
    log_x=False                   # Se puede cambiar a True si los años están muy agrupados
)

# 3. Personalización y Estilo del Gráfico
fig_bubble.update_layout(
    title='<b>Sostenibilidad vs. Antigüedad Empresarial</b><br><sup>El tamaño de la burbuja indica los ingresos operacionales</sup>',
    title_x=0.5,
    xaxis_title="Año de Fundación",
    yaxis_title="Puntaje Total de Sostenibilidad",
    font=dict(
        family="Arial, sans-serif",
        size=14,
        color="black"
    ),
    legend_title_text='Macrosector'
)

# Personalización del hover para mostrar la información de forma más clara
fig_bubble.update_traces(
    hovertemplate='<b>%{hovertext}</b><br><br>' +
                  'Año de Fundación: %{x}<br>' +
                  'Puntaje Sostenibilidad: %{y:.2f}<br>' +
                  'Ingresos Operacionales: %{marker.size:,.0f}<extra></extra>'
)

fig_bubble.show()
save_chart_as_html(fig_bubble, '05_bubble_chart_sustainability_vs_age.html')

## 6. Gráfico de Coordenadas Paralelas para Perfiles de Sostenibilidad

In [86]:
# 1. Preparación de Datos: Crear una tabla pivotada (ancha)
# Se agrupan los datos por empresa y pilar, sumando la valoración ponderada.
pivoted_data = raw_data.groupby(['Razón social', 'Nombre Pilar'])['valoracionPonderada'].sum().reset_index()

# Se pivota la tabla para que las empresas sean filas y los pilares, columnas.
# Los valores NaN se rellenan con 0, asumiendo desempeño nulo en ese pilar.
parallel_coords_df = pivoted_data.pivot(
    index='Razón social',
    columns='Nombre Pilar',
    values='valoracionPonderada'
).fillna(0).reset_index()

# 2. Enriquecer los datos con información contextual (Macrosector y Puntaje Total)
# Se une la tabla pivotada con los datos agregados para poder colorear las líneas.
parallel_coords_df = pd.merge(
    parallel_coords_df,
    company_agg_data[['Razón social', 'Macrosector', 'Puntaje_Total_Sostenibilidad']],
    on='Razón social',
    how='left'
)

# 3. Creación del Gráfico de Coordenadas Paralelas
# Se definen las dimensiones del gráfico, que son las columnas de los pilares.
dimensions = list(parallel_coords_df.columns)
dimensions.remove('Razón social')
dimensions.remove('Macrosector')
dimensions.remove('Puntaje_Total_Sostenibilidad')

fig_parallel = px.parallel_coordinates(
    parallel_coords_df,
    # El color de la línea representa el puntaje total de sostenibilidad.
    color="Puntaje_Total_Sostenibilidad",
    dimensions=dimensions,
    # Se añade el Macrosector y el nombre de la empresa a la información del hover.
    labels={"Razón social": "Empresa", "Macrosector": "Sector"},
    color_continuous_scale=px.colors.sequential.Blues, # Escala de color de oscuro (bajo) a brillante (alto)
    title="Perfiles de Sostenibilidad por Empresa"
)

# 4. Personalización y Estilo del Gráfico
fig_parallel.update_layout(
    title='<b>Perfiles de Sostenibilidad por Empresa</b><br><sup>Cada línea es una empresa, coloreada por su puntaje total</sup><br>',
    title_x=0.5,
    title_y=0.95,
    font=dict(family="Arial, sans-serif", size=12, color="black")
)

fig_parallel.show()
save_chart_as_html(fig_parallel, '06_parallel_coordinates_sustainability_profiles.html')

## 7. Diagrama Sankey de Flujo de Valoración

In [87]:
# 1. Preparación de Datos para el Sankey
# Se necesita una lista de flujos: origen (source), destino (target) y valor (value).
# El flujo es: Bloque -> Nombre Pilar -> Macrosector

# Flujo 1: Bloque a Nombre Pilar
flow1 = raw_data.groupby(['Bloque', 'Nombre Pilar'])['valoracionPonderada'].sum().reset_index()
flow1.rename(columns={'Bloque': 'source', 'Nombre Pilar': 'target', 'valoracionPonderada': 'value'}, inplace=True)

# Flujo 2: Nombre Pilar a Macrosector
flow2 = raw_data.groupby(['Nombre Pilar', 'Macrosector'])['valoracionPonderada'].sum().reset_index()
flow2.rename(columns={'Nombre Pilar': 'source', 'Macrosector': 'target', 'valoracionPonderada': 'value'}, inplace=True)

# Combinar ambos flujos en un único DataFrame
sankey_data = pd.concat([flow1, flow2], axis=0)

# Filtrar flujos con valor cero para no saturar el gráfico
sankey_data = sankey_data[sankey_data['value'] > 0]

# Crear una lista única de todos los nodos (orígenes y destinos)
unique_nodes = pd.unique(sankey_data[['source', 'target']].values.ravel('K'))

# Crear un diccionario para mapear cada nodo a un índice numérico
node_mapping = {node: i for i, node in enumerate(unique_nodes)}

# Reemplazar los nombres de los nodos por sus índices en el DataFrame de flujos
sankey_data['source_id'] = sankey_data['source'].map(node_mapping)
sankey_data['target_id'] = sankey_data['target'].map(node_mapping)

# 2. Creación del Gráfico Sankey con Plotly Graph Objects
fig_sankey = go.Figure(data=[go.Sankey(
    node=dict(
      pad=15,
      thickness=20,
      line=dict(color="black", width=0.5),
      label=unique_nodes,
    ),
    link=dict(
      source=sankey_data['source_id'],
      target=sankey_data['target_id'],
      value=sankey_data['value']
  ))])

# 3. Personalización y Estilo del Gráfico
fig_sankey.update_layout(
    title_text="<b>Diagrama Sankey del Flujo de Valoración de Sostenibilidad</b><br><sup>Flujo desde Bloque -> Pilar -> Macrosector</sup>",
    title_x=0.5,
    font=dict(family="Arial, sans-serif", size=12, color="black")
)

fig_sankey.show()
save_chart_as_html(fig_sankey, '07_sankey_diagram_sustainability_flows.html')


## 8. Gráfico Solar (Sunburst) de la Jerarquía del Desempeño

In [88]:
# 1. Preparación de Datos para el Sunburst
# No se requiere una gran preparación, Plotly Express puede manejar la jerarquía directamente.
# Se asegura que no haya valores nulos en las columnas de la jerarquía.
sunburst_data = raw_data.dropna(subset=['Bloque', 'Nombre Pilar'])
sunburst_data = sunburst_data[sunburst_data['valoracionPonderada'] > 0]

# 2. Creación del Gráfico Sunburst con Plotly Express
fig_sunburst = px.sunburst(
    sunburst_data,
    # La jerarquía define los anillos del gráfico, desde el centro hacia afuera.
    path=[px.Constant("Desempeño Total"), 'Bloque', 'Nombre Pilar'],
    # El tamaño y el color de cada segmento se basan en la suma de la valoración ponderada.
    values='valoracionPonderada',
    color='valoracionPonderada',
    # Se usa una escala de color que va de un color claro a uno oscuro para representar la magnitud.
    color_continuous_scale='Blues',
    hover_data={'Bloque': True, 'Nombre Pilar': True}
)

# 3. Personalización y Estilo del Gráfico
fig_sunburst.update_layout(
    title_text="<b>Gráfico Solar de la Jerarquía del Desempeño en Sostenibilidad</b><br><sup>Tamaño y color representan la contribución de cada área</sup>",
    title_x=0.5,
    margin = dict(t=60, l=25, r=25, b=25),
    font=dict(family="Arial, sans-serif", size=12, color="black")
)

# Personalización del hover para mostrar la información de forma más clara
fig_sunburst.update_traces(
    hovertemplate='<b>%{label}</b><br>Valoración Ponderada Total: %{value:,.2f}<br>Contribución al Padre: %{percentParent:.2%}<extra></extra>'
)

fig_sunburst.show()
save_chart_as_html(fig_sunburst, '08_sunburst_performance_hierarchy.html')

## 9. Matriz de Correlación entre Pilares de Sostenibilidad

In [89]:
# --- 9. Matriz de Correlación entre Pilares de Sostenibilidad ---
pillar_data = parallel_coords_df[dimensions]
correlation_matrix = pillar_data.corr()
fig_corr_matrix = px.imshow(
    correlation_matrix,
    text_auto=True,
    aspect="auto",
    color_continuous_scale='blues', 
    zmin=-1, zmax=1
)
fig_corr_matrix.update_layout(
    title_text='<b>Matriz de Correlación entre Pilares de Sostenibilidad</b><br><sup>Revela sinergias (azul) y trade-offs (rojo)</sup>',
    title_x=0.5,
    xaxis_tickangle=-45,
    font=dict(family="Arial, sans-serif", size=10, color="black")
)
fig_corr_matrix.show()
save_chart_as_html(fig_corr_matrix, '09_correlation_matrix_sustainability_pillars.html')

## 10. Gráfico de Barras Divergentes: Desempeño Relativo al Sector

In [90]:
sector_avg_score = company_agg_data.groupby('Macrosector')['Puntaje_Total_Sostenibilidad'].mean().reset_index()
sector_avg_score.rename(columns={'Puntaje_Total_Sostenibilidad': 'Promedio_Sector'}, inplace=True)
diverging_data = pd.merge(company_agg_data, sector_avg_score, on='Macrosector')
diverging_data['Diferencia_vs_Promedio'] = diverging_data['Puntaje_Total_Sostenibilidad'] - diverging_data['Promedio_Sector']
diverging_data['Desempeño_Relativo'] = np.where(diverging_data['Diferencia_vs_Promedio'] >= 0, 'Superior al Promedio', 'Inferior al Promedio')
diverging_data.sort_values(by=['Macrosector', 'Diferencia_vs_Promedio'], inplace=True)
fig_diverging_bar = px.bar(
    diverging_data,
    x='Diferencia_vs_Promedio',
    y='Razón social',
    color='Desempeño_Relativo',
    color_discrete_map={
        'Superior al Promedio': 'green',
        'Inferior al Promedio': 'red'
    },
    orientation='h',
    labels={'Diferencia_vs_Promedio': 'Diferencia vs. Promedio del Sector', 'Razón social': 'Empresa'}
)
fig_diverging_bar.update_layout(
    title='<b>Desempeño Relativo de Sostenibilidad vs. Promedio del Sector</b><br><sup>Barras verdes superan el promedio, rojas están por debajo</sup>',
    title_x=0.5,
    yaxis_title='Empresa',
    xaxis_title='Desviación del Promedio del Sector',
    font=dict(family="Arial, sans-serif", size=10, color="black"),
    height=max(600, len(diverging_data) * 20) 
)
fig_diverging_bar.show()
save_chart_as_html(fig_diverging_bar, '09_diverging_bar_performance_vs_sector.html')

# **Parte 3: Análisis Comparativo y de Atributos Específicos**

## 11. Gráfico de Cajas Comparativo: Propiedad y Desempeño (Pública vs. Privada)

In [91]:
property_type_data = raw_data[['Razón social', 'Tipo de propiedad (Privada, Pública, Mixta)']].drop_duplicates()
boxplot_data = pd.merge(company_agg_data, property_type_data, on='Razón social')
boxplot_data.dropna(subset=['Tipo de propiedad (Privada, Pública, Mixta)'], inplace=True)
fig_box_plot = px.box(
    boxplot_data,
    x='Tipo de propiedad (Privada, Pública, Mixta)',
    y='Puntaje_Total_Sostenibilidad',
    color='Tipo de propiedad (Privada, Pública, Mixta)',
    notched=True,
    points="all"
)
fig_box_plot.update_layout(
    title='<b>Comparativa de Desempeño en Sostenibilidad por Tipo de Propiedad</b>',
    title_x=0.5,
    xaxis_title='Tipo de Propiedad de la Empresa',
    yaxis_title='Puntaje Total de Sostenibilidad',
    font=dict(family="Arial, sans-serif", size=12, color="black"),
    showlegend=False
)
fig_box_plot.show()
save_chart_as_html(fig_box_plot, '11_boxplot_performance_by_property_type.html')

## 12. Gráfico de Densidad por Atributo: Multinacional vs. Nacional

In [92]:
# 1. Preparación de Datos
multinational_info = raw_data[['Razón social', '¿Multinacional? Si/No']].drop_duplicates()
density_data = pd.merge(company_agg_data, multinational_info, on='Razón social')
density_data.dropna(subset=['¿Multinacional? Si/No'], inplace=True)

# Definir grupos y colores
groups = {
    'Multinacional': 'Si',
    'Nacional': 'No'
}
colors = ['#1f77b4', '#ff7f0e']

# 2. Creación del Gráfico de Densidad con Scipy y Plotly GO
fig_density = go.Figure()

for (group_name, group_id), color in zip(groups.items(), colors):
    # Filtrar datos para el grupo actual
    current_data = density_data[density_data['¿Multinacional? Si/No'] == group_id]['Puntaje_Total_Sostenibilidad']
    
    if len(current_data) > 1:
        # Calcular KDE usando scipy
        kde = gaussian_kde(current_data)
        
        # Crear un rango de valores X para la curva
        x_range = np.linspace(current_data.min(), current_data.max(), 500)
        
        # Calcular los valores Y de la curva de densidad
        y_values = kde(x_range)
        
        # Añadir la curva al gráfico
        fig_density.add_trace(go.Scatter(
            x=x_range, 
            y=y_values, 
            mode='lines', 
            name=group_name,
            line=dict(color=color),
            fill='tozeroy'
        ))

# 3. Personalización y Estilo del Gráfico
fig_density.update_layout(
    title_text='<b>Distribución del Desempeño: Multinacional vs. Nacional</b>',
    title_x=0.5,
    xaxis_title='Puntaje Total de Sostenibilidad',
    yaxis_title='Densidad',
    font=dict(family="Arial, sans-serif", size=12, color="black"),
    legend_title_text='Tipo de Empresa'
)

fig_density.show()

save_chart_as_html(fig_density, '12_density_plot_multinational_vs_national.html')


## 13. Análisis de Foco Temático: Comparativa de Variables entre Líderes y Rezagados

Descripción Detallada:

Un mapa de calor (heatmap) que despliega todas las Variables de sostenibilidad en el eje Y y todos los Macrosector en el eje X. La intensidad del color de cada celda (de un color claro para valores bajos a uno oscuro e intenso para valores altos) representa la valoración ponderada promedio que las empresas de un sector obtienen en una variable específica. A diferencia de solo contar la frecuencia, este enfoque mide el "impacto" o la "contribución al puntaje" de cada variable dentro de su contexto sectorial.

Análisis Estratégico Potencial:

Esta visualización ofrece un diagnóstico profundo sobre dónde reside el verdadero valor de las acciones de sostenibilidad, permitiendo un análisis mucho más granular que la simple frecuencia de reporte.

Identificación de Focos de Valor Sectorial: Permite a un estratega identificar instantáneamente qué iniciativas específicas (variables) son las que más contribuyen al puntaje total en cada industria. Responde a preguntas clave como: ¿Es más valioso para el sector 'Manufactura' enfocarse en 'Reducción de emisiones' o en 'Seguridad laboral'? ¿Dónde se concentran los puntajes más altos en el sector 'Servicios'?

In [101]:
# 1. Preparación de Datos
# Calcular la valoración ponderada promedio para cada variable dentro de cada macrosector.
var_importance_data = raw_data.groupby(['Macrosector', 'Variable'])['valoracionPonderada'].mean().reset_index()

# Pivotar la tabla para crear una matriz: Variables en filas, Sectores en columnas.
var_importance_matrix = var_importance_data.pivot(
    index='Variable', 
    columns='Macrosector', 
    values='valoracionPonderada'
).fillna(0)

# 2. Creación del Gráfico (Heatmap)
fig_var_importance = px.imshow(
    var_importance_matrix,
    text_auto=".2f", # Formatea los números a dos decimales
    aspect="auto",
    color_continuous_scale='Blues', # Una escala de color secuencial es más apropiada aquí
    labels=dict(x="Macrosector", y="Variable de Sostenibilidad", color="Importancia Promedio")
)

# 3. Personalización y Estilo del Gráfico
fig_var_importance.update_layout(
    title_text='<b>Importancia Relativa de Variables por Macrosector</b><br><sup>El color representa la contribución promedio de cada variable al puntaje del sector</sup>',
    title_x=0.5,
    xaxis_tickangle=-45,
    font=dict(family="Arial, sans-serif", size=10, color="black"),
    height=max(600, len(var_importance_matrix.index) * 20) # Ajustar altura dinámicamente
)
fig_var_importance.show()
save_chart_as_html(fig_var_importance, '13_variable_importance_heatmap.html')

## 14. Gráfico de Dispersión con Línea de Tendencia: Ingresos vs. Valoración Ponderada

In [97]:
# 1. Preparación de Datos
# Reutilizamos el DataFrame agregado, filtrando valores no positivos en ingresos para la escala logarítmica.
scatter_data = company_agg_data[company_agg_data['Ingresos_Operacionales'] > 0].copy()

# 2. Creación del Gráfico de Dispersión
fig_scatter_trend = px.scatter(
    scatter_data,
    x="Ingresos_Operacionales",
    y="Puntaje_Total_Sostenibilidad",
    color="Macrosector",
    hover_name="Razón social",
    log_x=True,  # Usar escala logarítmica para el eje X (Ingresos)
    trendline="ols",  # Añadir línea de tendencia de regresión (Ordinary Least Squares)
    trendline_scope="overall" # Una única línea de tendencia para todos los datos
)

# 3. Personalización y Estilo del Gráfico
fig_scatter_trend.update_layout(
    title='<b>Relación entre Ingresos Operacionales y Desempeño en Sostenibilidad</b>',
    title_x=0.5,
    xaxis_title='Ingresos Operacionales (Escala Logarítmica)',
    yaxis_title='Puntaje Total de Sostenibilidad',
    font=dict(family="Arial, sans-serif", size=12, color="black"),
    legend_title_text='Macrosector'
)

fig_scatter_trend.show()
save_chart_as_html(fig_scatter_trend, '14_scatter_trend_income_vs_sustainability.html')

## 15. Diagrama de Cuerdas (Chord Diagram) de Interconexión Sector-Pilar

In [99]:
# 1. Preparación de Datos
# Agrupar por Macrosector y Nombre Pilar, sumando la valoración ponderada.
chord_data = raw_data.groupby(['Macrosector', 'Nombre Pilar'])['valoracionPonderada'].sum().reset_index()

# Pivotar los datos para crear una matriz: Sectores en filas, Pilares en columnas.
chord_matrix = chord_data.pivot(
    index='Macrosector', 
    columns='Nombre Pilar', 
    values='valoracionPonderada'
).fillna(0)

# 2. Creación del Gráfico (Heatmap)
fig_chord_alternative = px.imshow(
    chord_matrix,
    text_auto=True,
    aspect="auto",
    color_continuous_scale='Viridis',
    labels=dict(x="Pilar de Sostenibilidad", y="Macrosector", color="Valoración Total")
)

# 3. Personalización y Estilo del Gráfico
fig_chord_alternative.update_layout(
    title_text='<b>Interconexión y Especialización: Macrosector vs. Pilar de Sostenibilidad</b><br><sup>El color representa la valoración total acumulada</sup>',
    title_x=0.5,
    xaxis_tickangle=-45,
    font=dict(family="Arial, sans-serif", size=10, color="black")
)

fig_chord_alternative.show()
save_chart_as_html(fig_chord_alternative, '15_chord_alternative_macrosector_vs_pillar.html')

## 16. Gráfico de Barras Anidadas: Variables Clave por Pilar y Liderazgo Sectorial

Descripción Detallada:

Esta visualización consiste en una serie de subgráficos (facetas), donde cada subgráfico representa un Nombre Pilar de sostenibilidad (ej. "Acción Climática", "Salud y Seguridad", etc.). Dentro de cada uno de estos subgráficos, se presenta un gráfico de barras horizontales que muestra las Variables más relevantes para ese pilar, ordenadas de mayor a menor según su valoración ponderada promedio.

La característica distintiva es que el color de cada barra no es estático, sino que representa el Macrosector que ha obtenido la puntuación más alta para esa variable específica dentro de ese pilar. De esta forma, el color de la barra responde a la pregunta: "¿Qué sector industrial lidera el desempeño en esta variable particular?".

Análisis Estratégico Potencial:

Esta visualización es una herramienta de diagnóstico táctico que permite desglosar el desempeño a un nivel muy detallado, revelando la dinámica competitiva dentro de cada pilar de sostenibilidad.

Identificación de "Motores de Valor" por Pilar: A diferencia de ver los pilares de forma agregada, este gráfico revela exactamente qué acciones (variables) son las que más contribuyen al éxito de un pilar. Permite a un estratega ver si el alto desempeño en "Acción Climática" se debe a la "Gestión de emisiones" o a la "Inversión en renovables".

Mapa de Liderazgo Táctico: Al colorear las barras por el macrosector líder, se crea un "mapa de fortalezas" muy claro. Se puede identificar si un sector, como 'Manufactura', domina consistentemente las variables dentro de los pilares ambientales, o si el liderazgo está distribuido, con el sector 'Servicios' liderando en variables de gobernanza dentro del mismo pilar.

In [122]:
# 1. Preparación de Datos
# Calcular el puntaje promedio de cada variable por sector y pilar.
df_agg = raw_data.groupby(['Nombre Pilar', 'Variable', 'Macrosector'])['valoracionPonderada'].mean().reset_index()

# Para cada variable en cada pilar, encontrar el sector líder (con el puntaje más alto).
idx = df_agg.groupby(['Nombre Pilar', 'Variable'])['valoracionPonderada'].idxmax()
df_leaders = df_agg.loc[idx][['Nombre Pilar', 'Variable', 'Macrosector']]
df_leaders.rename(columns={'Macrosector': 'Sector_Lider'}, inplace=True)

# Calcular el puntaje promedio general de cada variable por pilar para la longitud de la barra.
df_plot_data = df_agg.groupby(['Nombre Pilar', 'Variable'])['valoracionPonderada'].mean().reset_index()

# Unir la información del sector líder con los datos del gráfico.
df_plot_data = pd.merge(df_plot_data, df_leaders, on=['Nombre Pilar', 'Variable'])
# Reemplazar los nombres de las variables largas para mejorar la legibilidad en el gráfico.
df_plot_data['Variable'] = df_plot_data['Variable'].apply(lambda x: wrap_text(x, max_length=30))

# 2. Creación del Gráfico de Barras Anidadas
# Se usa facet_col para crear un subgráfico por cada 'Nombre Pilar'.
fig_nested_bars = px.bar(
    df_plot_data,
    x='valoracionPonderada',
    y='Variable',
    color='Sector_Lider',  # El color de la barra indica el sector líder.
    orientation='h',
    facet_col='Nombre Pilar',
    facet_col_wrap=3,  # Organiza los subgráficos en 2 columnas.
    labels={'valoracionPonderada': 'Valoración Ponderada Promedio', 'Variable': ''},
    color_discrete_sequence=px.colors.qualitative.Set2, # Añadir una paleta de colores
    facet_col_spacing=0.01  # Añadir espacio horizontal entre subgráficos
)

# 3. Personalización y Estilo del Gráfico
fig_nested_bars.update_layout(
    title_text='<b>Variables Clave por Pilar y Liderazgo Sectorial</b><br><sup>El color de la barra indica el Macrosector con mayor puntaje en esa variable</sup>',
    title_x=0.5,
    font=dict(family="Arial, sans-serif", size=10, color="black"),
    height=max(800, len(df_plot_data['Nombre Pilar'].unique()) * 300), # Altura dinámica
)

# Hacer que cada subgráfico tenga su propio eje Y independiente y ordenar las barras.
fig_nested_bars.update_yaxes(matches=None, showticklabels=True)
fig_nested_bars.update_yaxes(categoryorder="total ascending")
# Actualizar los títulos de cada subgráfico para que no muestren "Nombre Pilar=".
fig_nested_bars.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

fig_nested_bars.show()
save_chart_as_html(fig_nested_bars, '16_nested_bars_variables_by_pillar_and_sector.html')

In [128]:
# 1. Preparación de Datos
# Calcular el puntaje promedio de cada variable por sector y pilar.
df_agg = raw_data.groupby(['Nombre Pilar', 'Variable', 'Macrosector'])['valoracionPonderada'].mean().reset_index()

# Para cada variable en cada pilar, encontrar el sector líder (con el puntaje más alto).
idx = df_agg.groupby(['Nombre Pilar', 'Variable'])['valoracionPonderada'].idxmax()
df_leaders = df_agg.loc[idx][['Nombre Pilar', 'Variable', 'Macrosector']]
df_leaders.rename(columns={'Macrosector': 'Sector_Lider'}, inplace=True)

# Calcular el puntaje promedio general de cada variable por pilar para la posición y tamaño del punto.
df_plot_data = df_agg.groupby(['Nombre Pilar', 'Variable'])['valoracionPonderada'].mean().reset_index()

# Unir la información del sector líder con los datos del gráfico.
df_plot_data = pd.merge(df_plot_data, df_leaders, on=['Nombre Pilar', 'Variable'])
# Reemplazar los nombres de las variables largas para mejorar la legibilidad en el gráfico.
df_plot_data['Variable'] = df_plot_data['Variable'].apply(lambda x: wrap_text(x, max_length=30))

# 2. Creación del Gráfico de Dispersión Anidado
# Se usa facet_col para crear un subgráfico por cada 'Nombre Pilar'.
fig_nested_scatter = px.scatter(
    df_plot_data,
    x='valoracionPonderada',
    y='Variable',
    size='valoracionPonderada', # El tamaño del punto también indica importancia.
    color='Sector_Lider',      # El color del punto indica el sector líder.
    hover_name='Sector_Lider',
    facet_col='Nombre Pilar',
    facet_col_wrap=3,          # Organiza los subgráficos en 2 columnas.
    labels={'valoracionPonderada': 'Valoración Ponderada Promedio', 'Variable': ''},
        facet_col_spacing=0.15
)

# 3. Personalización y Estilo del Gráfico
fig_nested_scatter.update_layout(
    title_text='<b>Variables Clave por Pilar y Liderazgo Sectorial</b><br><sup>El color indica el Macrosector líder; el tamaño, la importancia de la variable</sup>',
    title_x=0.5,
    font=dict(family="Arial, sans-serif", size=10, color="black"),
    height=max(800, len(df_plot_data['Nombre Pilar'].unique()) * 200), # Altura dinámica

)

# Hacer que cada subgráfico tenga su propio eje Y independiente y ordenar las variables.
fig_nested_scatter.update_yaxes(matches=None, showticklabels=True)
fig_nested_scatter.update_yaxes(categoryorder="total ascending")
# Actualizar los títulos de cada subgráfico para que no muestren "Nombre Pilar=".
fig_nested_scatter.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

fig_nested_scatter.show()